# Imports
Import required libraries

In [132]:
from __future__ import print_function, division
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn as sklearn
from numpy import random
import sqlite3
%matplotlib inline

# 1) Initializing the dataset
Read-in the tables from the sqlite database.

In [133]:
with sqlite3.connect('database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    matches = pd.read_sql_query("SELECT * from Match", con)


For now, we will currently only be examining betting data from bet365. Many of the betting sites included in the dataset are missing betting data on a large portion of the matches and bet365 has by far the most.

We will also drop attributes that will not be useful for what we want yet; these attributes are generally ingame statistics of the matches in general.

In [134]:
# Drops unneeded attributes
atts_to_drop = ['BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'GBD',
               'PSH', 'PSD', 'PSA', 'WHH', 'WHA', 'WHD', 'SJH', 'SJD', 'SJA', 'VCH',
               'VCD', 'VCA', 'GBH', 'GBA', 'BSH', 'BSD', 'BSA', 'goal', 'shoton', 'shotoff',
               'foulcommit', 'card', 'cross', 'corner', 'possession']
matches = matches.drop(atts_to_drop, axis=1) 

# Drops player formation values
matches = matches.drop(matches.columns[11:55], axis=1)

# Drops null values
matches = matches.dropna()

Splits the matches data based on their country. Often times countries will have many different leagues, but this dataset only includes data from the biggest league in each country.
We will only be using data from the top six biggest European leagues.

In [135]:
print(countries)
england_matches = matches[(matches['country_id'] == 1729)]
france_matches = matches[(matches['country_id'] == 4769)]
germany_matches = matches[(matches['country_id'] == 7809)]
italy_matches = matches[(matches['country_id'] == 10257)]
netherlands_matches = matches[(matches['country_id'] == 13274)]
spain_matches = matches[(matches['country_id'] == 21518)]

       id         name
0       1      Belgium
1    1729      England
2    4769       France
3    7809      Germany
4   10257        Italy
5   13274  Netherlands
6   15722       Poland
7   17642     Portugal
8   19694     Scotland
9   21518        Spain
10  24558  Switzerland


# 2) Ranking the Leagues by Ease of Prediction
Fans of soccer know that certain leagues are easier to predict than others. For example, the Spanish La Liga is notoriously top-heavy, with Real Madrid and Barcelona dominating the league for the last few decades. Let's see if we can b